# Evaluation of the agent (VisualAgent)

In [1]:
%pip  install litellm

Note: you may need to restart the kernel to use updated packages.


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# === Base ===
import os
import json
import pandas as pd
from tqdm import tqdm

# === Phoenix core ===
import phoenix as px
from phoenix.trace import SpanEvaluations
from phoenix.trace.dsl import SpanQuery

# === Evaluaciones automáticas ===
from phoenix.evals import (
    TOOL_CALLING_PROMPT_TEMPLATE,
    llm_classify,
    PromptTemplate,
)


# === LLM local (ej. llama.cpp o llamafile) ===
#from phoenix.evals.models import 

# === Extra ===
from openinference.instrumentation import suppress_tracing
import nest_asyncio
nest_asyncio.apply()
import pprint

In [3]:
os.environ['OLLAMA_API_BASE']= 'http://localhost:11434'

In [4]:
PROJECT_NAME = "evaluating-agent"

In [5]:
from utils import run_graph_with_tracing, start_main_span


<module 'langgraph.version' from 'c:\\Users\\david\\Documents\\LangGraph\\LangChainCourse\\lc-academy-env\\Lib\\site-packages\\langgraph\\version.py'>
OpenTelemetry Tracing Details
|  Phoenix Project: evaluating-agent
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [6]:
'''from phoenix.evals.models import BaseModel
from typing import Optional, Sequence, Union, Any
from langchain_ollama import ChatOllama
import asyncio'
'''

from phoenix.evals import llm_classify, TOOL_CALLING_PROMPT_TEMPLATE, PromptTemplate, LiteLLMModel
from litellm import completion


In [9]:
# === Correct Phoenix Trace Querying ===
from phoenix.trace import SpanEvaluations

# Define evaluation queries properly
sql_query = (
    SpanQuery()
    .where("name == 'sql_query_exec' and span_kind == 'TOOL'")
).select(
    question="input.value",
    query_gen="output.value",
)

analysis_query = (
    SpanQuery()
    .where("name == 'data_analysis' and span_kind == 'TOOL'")
).select(
    query="input.value",
    response="output.value",
)

viz_query = (
    SpanQuery()
    .where("name == 'gen_visualization' and span_kind == 'TOOL'")
).select(
    input="input.value",
    generated_code="output.value",
)

decide_query = (
    SpanQuery()
    .where("span_kind == 'TOOL' and name == 'decide_tool'")
).select(
    question="input.value",
    tool_call="output.value",
)

Create loop to see multiple times the difference in the "re-runs"
1. Use multiple type of prompts styles for testing
2. Human in the loop?

1. complete
2. How to save a csv file with the output of the traces.
3. Try to make a decision based on different branches and parallelization
4. cook up a docker container
5. Try access with the server

In [8]:
agent_questions = [
    "What was the most popular product SKU?",
    "What was the total revenue across all stores?",
    "Which store had the highest sales volume?",
    "Create a bar chart showing total sales by store",
    "What was the average transaction value?"
]

for question in tqdm(agent_questions, desc="Processing questions"):
    try:
        input_state = {
            "prompt": question,
        }
        ret = run_graph_with_tracing(input_state)
    except Exception as e:
        print(f"Error processing question: {question}")
        print(e)
        continue

Processing questions:   0%|          | 0/5 [00:00<?, ?it/s]

🔁 Starting LangGraph execution with tracing
Elección de herramienta: lookup_sales_data
What was the most popular product SKU?
Elección de herramienta: analyzing_data
Elección de herramienta: create_visualization
Elección de herramienta: analyzing_data
Elección de herramienta: end
✅ LangGraph execution completed


Processing questions:  20%|██        | 1/5 [00:30<02:01, 30.48s/it]

🔁 Starting LangGraph execution with tracing
Elección de herramienta: lookup_sales_data
What was the total revenue across all stores?
Elección de herramienta: analyzing_data
Elección de herramienta: analyzing_data
Elección de herramienta: end
✅ LangGraph execution completed


Processing questions:  40%|████      | 2/5 [00:46<01:06, 22.07s/it]

...

✅ LangGraph execution completed
🔁 Starting LangGraph execution with tracing
Elección de herramienta: lookup_sales_data
Create a bar chart showing total sales by store
Elección de herramienta: create_visualization
Elección de herramienta: analyzing_data
Elección de herramienta: create_visualization
Elección de herramienta: analyzing_data
Elección de herramienta: end


Processing questions:  80%|████████  | 4/5 [01:50<00:29, 29.78s/it]

✅ LangGraph execution completed
🔁 Starting LangGraph execution with tracing
Elección de herramienta: lookup_sales_data
What was the average transaction value?
Elección de herramienta: analyzing_data
Elección de herramienta: analyzing_data
Elección de herramienta: end


Processing questions: 100%|██████████| 5/5 [02:00<00:00, 24.09s/it]

✅ LangGraph execution completed


### Prompts for evaluation of the tools


In [16]:
# === SQL Generation Evaluation ===
SQL_EVAL_GEN_PROMPT = """
SQL Evaluation Prompt:
-----------------------
You are tasked with determining if the SQL generated appropiately answers a given instruction
taking into account its generated query and response.

Data:
-----
- [Instruction]: {question}
  This section contains the specific task or problem that the sql query is intended to solve.

- [Reference Query]: {query_gen}
  This is the sql query submitted for evaluation. Analyze it in the context of the provided
  instruction.

Evaluation:
-----------
Your response should be a single word: either "correct" or "incorrect".
You must assume that the db exists and that columns are appropiately named.
You must take into account the response as additional information to determine the correctness.

- "correct" indicates that the sql query correctly solves the instruction.
- "incorrect" indicates that the sql query correctly does not solve the instruction correctly.

Note: Your response should contain only the word "correct" or "incorrect" with no additional text
or characters.
"""

# === Data Analysis Evaluation ===
CLARITY_LLM_JUDGE_PROMPT = """
In this task, you will be presented with a query and an answer. Your objective is to evaluate the clarity 
of the answer in addressing the query. A clear response is one that is precise, coherent, and directly 
addresses the query without introducing unnecessary complexity or ambiguity. An unclear response is one 
that is vague, disorganized, or difficult to understand, even if it may be factually correct.

Your response should be a single word: either "clear" or "unclear," and it should not include any other 
text or characters. "clear" indicates that the answer is well-structured, easy to understand, and 
appropriately addresses the query. "unclear" indicates that some part of the response could be better 
structured or worded.
Please carefully consider the query and answer before determining your response.

After analyzing the query and the answer, you must write a detailed explanation of your reasoning to 
justify why you chose either "clear" or "unclear." Avoid stating the final label at the beginning of your 
explanation. Your reasoning should include specific points about how the answer does or does not meet the 
criteria for clarity.

[BEGIN DATA]
Query: {query}
Answer: {response}
[END DATA]
Please analyze the data carefully and provide an explanation followed by your response.

EXPLANATION: Provide your reasoning step by step, evaluating the clarity of the answer based on the query.
LABEL: "clear" or "unclear"
"""

# === Visualization Evaluation ===
VIZ_QUALITY_TEMPLATE = PromptTemplate("""
Evaluate this visualization configuration:
1. Appropriateness of chart type for the data
2. Correct mapping of axes
3. Clarity of visualization goal

Goal: {input}
Data Sample: {reference_data}
Configuration: {output}

Respond with "good" or "poor" and a brief reason.
""")

In [13]:
tools = [
    {
        "name": "lookup_sales_data",
        "description": "Fetch historical data of sales for a product or category."
    },
    {
        "name": "analyzing_data",
        "description": "Does a statistical analysis of the data available, giving an output in form of a summary of trends/patterns found for example."
    },
    {
        "name": "create_visualization",
        "description": "Generates a visualization schema of the data processed according to the user's configuration."
    }
]


#### Evaluators 

In [ ]:
from opentelemetry.sdk.trace.export import SimpleSpanProcessor, ConsoleSpanExporter
from opentelemetry import trace

# Añade exportador a consola para debug
trace.get_tracer_provider().add_span_processor(
    SimpleSpanProcessor(ConsoleSpanExporter())
)

In [36]:
print(TOOL_CALLING_PROMPT_TEMPLATE)


You are an evaluation assistant evaluating questions and tool calls to
determine whether the tool called would answer the question. The tool
calls have been generated by a separate agent, and chosen from the list of
tools provided below. It is your job to decide whether that agent chose
the right tool to call.

    [BEGIN DATA]
    ************
    [Question]: {question}
    ************
    [Tool Called]: {tool_call}
    [END DATA]

Your response must be single word, either "correct" or "incorrect",
and should not contain any text or characters aside from that word.
"incorrect" means that the chosen tool would not answer the question,
the tool includes information that is not presented in the question,
or that the tool signature includes parameter values that don't match
the formats specified in the tool signatures below.

"correct" means the correct tool call was chosen, the correct parameters
were extracted from the question, the tool call generated is runnable and correct,
and tha

In [20]:
from litellm import completion
#model = completion(model='ollama_chat/llama2', api_base="http://localhost:11434", stream=False)
#model.reload_client = lambda: None
##model.verbose_generation_info = lambda: ""
#model._timeout = lambda: 0
model = LiteLLMModel(model="ollama_chat/llama3.2:3B")
input_state = {"prompt": "Show me sales in Nov 2021"}
#result = run_graph_with_tracing(input_state)
#pprint.pprint(result)

#verify traces

tool_calls_df = px.Client().query_spans(decide_query, project_name=PROJECT_NAME, timeout=None)
tool_calls_df = tool_calls_df.dropna(subset=["tool_call"])

tool_calls_df.head()

tool_call_eval = llm_classify(
    dataframe=tool_calls_df,
    template=TOOL_CALLING_PROMPT_TEMPLATE.template[0].template.replace(
        "{tool_definitions}", json.dumps(tools).replace("{", '"').replace("}", '"')),
    rails=['correct', 'incorrect'],
    model=model,
    provide_explanation=True,
    concurrency=1,
)

tool_call_eval['score'] = tool_call_eval.apply(lambda x: 1 if x['label']=='correct' else 0, axis=1)

tool_call_eval.head()


llm_classify |          | 0/73 (0.0%) | ⏳ 00:00<? | ?it/s

,label,explanation,exceptions,execution_status,execution_seconds,score
context.span_id,,,,,,
faa0113f472fb26a,NOT_PARSABLE,incorrect,[],COMPLETED,2.919449,0
522a99763361f627,NOT_PARSABLE,incorrect,[],COMPLETED,0.445795,0
5193ea1de96a7e4f,NOT_PARSABLE,incorrect,[],COMPLETED,2.435714,0
53a22c3a269b96d4,NOT_PARSABLE,incorrect,[],COMPLETED,0.412020,0
229a2f9084e4d308,NOT_PARSABLE,correct,[],COMPLETED,0.247947,0


In [21]:
px.Client().log_evaluations(
    SpanEvaluations(eval_name="Tool Calling Eval", dataframe=tool_call_eval),
)

In [31]:
sql_df = px.Client().query_spans(sql_query, project_name=PROJECT_NAME, timeout=None)
sql_df.head()

,question,query_gen
context.span_id,,
f336001a171b6854,What was the most popular product SKU?,SKU_Coded\n0 6200700
ca53fe2ff8052d77,What was the total revenue across all stores?,sum(Total_Sale_Value)\n0 1.327264...
87feb9c9d85bad98,Which store had the highest sales volume?,Store_Number\n0 2970
5f64e036ee56ba02,Which store had the highest sales volume?,Store_Number\n0 2970
b8cb9cb04f90ddc6,Create a bar chart showing total sales by store,Store_Number sum(Total_Sale_Value)\n0 ...


In [30]:
query = (SpanQuery()
    .where("name == 'sql_query_exec' and span_kind == 'TOOL'")
).select(
    question="input.value",
    query_gen="output.value",
)
query_df = px.Client().query_spans(query, project_name=PROJECT_NAME, timeout=None)
query_df.head()

,question,query_gen
context.span_id,,
f336001a171b6854,What was the most popular product SKU?,SKU_Coded\n0 6200700
ca53fe2ff8052d77,What was the total revenue across all stores?,sum(Total_Sale_Value)\n0 1.327264...
87feb9c9d85bad98,Which store had the highest sales volume?,Store_Number\n0 2970
5f64e036ee56ba02,Which store had the highest sales volume?,Store_Number\n0 2970
b8cb9cb04f90ddc6,Create a bar chart showing total sales by store,Store_Number sum(Total_Sale_Value)\n0 ...


#### Evaluator for the other tools


In [32]:
# === SQL Generation Evaluation ===

sql_df = px.Client().query_spans(sql_query, project_name=PROJECT_NAME, timeout=None)
#sql_df = sql_df[sql_df["question"].str.contains("Generate an SQL query based on a prompt.", na=False)]

with suppress_tracing():
    sql_eval = llm_classify(
        dataframe=sql_df,
        template=SQL_EVAL_GEN_PROMPT,
        rails=["correct", "incorrect"],
        model=model,
        provide_explanation=True
    )

sql_eval ['score'] = sql_eval.apply(lambda x: 1 if x['label']=='correct' else 0, axis=1)
sql_eval.head()
px.Client().log_evaluations(
    SpanEvaluations(eval_name="SQL Generation Eval", dataframe=sql_eval),
)

llm_classify |          | 0/6 (0.0%) | ⏳ 00:00<? | ?it/s

In [26]:
sql_df.head()

,question,query_gen
context.span_id,,


In [22]:
# === Data Analysis Evaluation ===
clarity_df = px.Client().query_spans(analysis_query, project_name=PROJECT_NAME, timeout=None)
clarity_df.head()
with suppress_tracing():
    clarity_eval = llm_classify(
        dataframe=clarity_df,
        template=CLARITY_LLM_JUDGE_PROMPT,
        rails=["clear", "unclear"],
        model=model,
        provide_explanation=True
    )
clarity_eval['score'] = clarity_eval.apply(lambda x: 1 if x['label']=='clear' else 0, axis=1)

clarity_eval.head()

px.Client().log_evaluations(
    SpanEvaluations(eval_name="Response Clarity", dataframe=clarity_eval),
)

llm_classify |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s

In [34]:
# === Visualization Evaluation ===
code_gen_df = px.Client().query_spans(viz_query, project_name=PROJECT_NAME, timeout=None)
code_gen_df.head()

def code_is_runnable(output:str) -> bool:
    output = output.replace("```python", "").replace("```", "").strip()
    try:
        exec(output)
        return True
    except Exception as e:
        return False
    
code_gen_df['label'] = code_gen_df['generated_code'].apply(code_is_runnable).map({True: "runnable", False: "not_runnable"})
code_gen_df['score'] = code_gen_df['label'].apply(lambda x: 1 if x=='runnable' else 0)

In [35]:
code_gen_df.head()

,input,generated_code,label,score
context.span_id,,,,


In [36]:
px.Client().log_evaluations(
    SpanEvaluations(eval_name="Runnable Code Eval", dataframe=code_gen_df),
)